In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('dark')
import numpy as np
import math
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,10)

# Canada Case and Death Data by Age group and Gender

The figures below show data from [StatsCan's Detailed Confirmed Cases](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1310076601) dataset via Jens von Bergmann's [CanCovidData R Package](https://github.com/mountainMath/CanCovidData). The total numbers of cases are much less (currently about 50%) of the more widely reported numbers, but they include much more detail including age and gender information.

First, use rpy2 to call the [CanCovidData/get_cansim_case_data](https://github.com/mountainMath/CanCovidData/blob/master/R/canada_covid_data_import.R#L303) as a pandas DataFrame.

In [ ]:
CanCovidData = importr("CanCovidData")
cansim_cases_r = CanCovidData.get_cansim_case_data();

with localconverter(ro.default_converter + pandas2ri.converter):
    CanSimDF = ro.conversion.rpy2py(cansim_cases_r)

### Cases by Age and Gender

Group by gender, then do value counts for each of the age group categories.

In [ ]:
caseDF = (CanSimDF.groupby('Gender', observed=True)['Age group'] 
          .value_counts()
          .unstack()
)

caseDF.index = caseDF.index.astype(str).sort_values()

ax = caseDF.plot(kind='bar', alpha=0.8)
ax.set_title('Stats Canada Reported Cases by Gender and Age Group')

### Death by Age and Gender

As above, but for those cases which in which the patient has died.

In [ ]:
deathDF = (CanSimDF[CanSimDF['Death'] == 'Yes'].groupby('Gender', observed=True)['Age group'] 
          .value_counts()
          .unstack()
)

deathDF.index = deathDF.index.astype(str).sort_values()

ax = deathDF.plot(kind='bar', alpha=0.8)
ax.set_title('Stats Canada Reported Deaths by Gender and Age Group')

## Population effects?

Women tend to live longer than men on average so what if we try to account for the gender distribution? Do we still see an excess of female cases?

In [ ]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [ ]:
# You should check if this name or location is stable or if it
# changes with new data releases
resp = urlopen("https://www150.statcan.gc.ca/n1/en/tbl/csv/17100005-eng.zip")
zipfile = ZipFile(BytesIO(resp.read()))

popDF = pd.read_csv(zipfile.open(zipfile.filelist[0]))


In [ ]:
year2019  = popDF['REF_DATE'] == 2019
countryCA = popDF['GEO'] == 'Canada'

popDF = popDF[year2019 & countryCA]

In [ ]:
# Recode into the groups used in the COVID19 data, 10 year bins
popGroups = {}

# Group together 0-4, 5-9, 10-14, 14-20
young_groups =  [f"{age_group} to {age_group + 4} years" for age_group in range(0, 20, 5)]
young_groups_c = [popDF['Age group'] == young_group for young_group in young_groups]
popGroups['0 to 19 years'] = pd.concat([popDF[group] for group in young_groups_c]).groupby('Sex').sum()

for age_group in range(20, 79, 10):
    low_age = f"{age_group} to {age_group + 4} years"
    hi_age  = f"{age_group + 5} to {age_group + 9} years"
    low = popDF['Age group'] == low_age
    hi  = popDF['Age group'] == hi_age
    popGroups[f"{age_group} to {age_group + 9} years"] = popDF[low | hi].groupby('Sex').sum()
    

# Group together 80-84, 85-89, 90-94, 94-99, 100+    
old_groups =  [f"{age_group} to {age_group + 4} years" 
               for age_group in range(80, 99, 5)] + ["100 years and over"]

old_groups_c = [popDF['Age group'] == old_group for old_group in old_groups]
popGroups['80 years or older'] = pd.concat([popDF[group] for group in old_groups_c]).groupby('Sex').sum()

In [ ]:
popDF2 = pd.concat(popGroups)['VALUE'].unstack().transpose().drop('Both sexes')
popDF2T = popDF2.transpose()


sns.set_style("dark")
fig, ax = plt.subplots()
bar_width=0.4

popDF2T = popDF2.transpose()
xtics = np.arange(len(popDF2T.index))
ax.bar(xtics, popDF2.loc['Females'], width=bar_width, alpha=0.4, color='red')
ax.bar(xtics + bar_width, popDF2.loc['Males'], width=bar_width, alpha=0.4, color='blue')


ax2 = ax.twinx()
ax2.plot(xtics + bar_width / 2, popDF2.loc['Males'] / popDF2.loc['Females'], marker='+', markersize=15)


ax.set_xticks(xtics + bar_width / 2)
ax.set_xticklabels(popDF2T.index)

ax2.set_ylabel('Male/Female ratio')
ax2.set_ylim([-1.5,1.5])
ax2.legend(["Population ratio (RHS scale)"])

ylabels = [f"{x/1e6:,.1f} M" for x in ax.get_yticks()]
ax.set_yticklabels(ylabels)
ax.set_ylabel('Population segment size')

In [ ]:
normCaseDF = caseDF.drop('Not stated', axis=1).drop(['Non-binary', 'Not stated'])
normCaseDF

In [ ]:
normCaseDF.loc['Female rate'] = normCaseDF.loc['Female'] / popDF2.loc['Females']
normCaseDF.loc['Male rate']   = normCaseDF.loc['Male'] / popDF2.loc['Males']
normCaseDF

In [ ]:
ax = normCaseDF.drop(['Female', 'Male'], axis=0).plot(kind='bar', alpha=0.8)
ax.set_title('Stats Canada Reported Cases by Gender and Age Group, normalized by pop segment size')
ax.set_ylabel('Fraction of population segment')

In [ ]:
normDeathDF = deathDF.drop('Not stated', axis=1).drop('Not stated')
normDeathDF

In [ ]:
normDeathDF.loc['Female rate'] = normDeathDF.loc['Female'] / popDF2.loc['Females']
normDeathDF.loc['Male rate']   = normDeathDF.loc['Male'] / popDF2.loc['Males']
normDeathDF

In [ ]:
ax = normDeathDF.drop(['Female', 'Male'], axis=0).plot(kind='bar', alpha=0.8)
ax.set_title('Stats Canada Reported Deaths by Gender and Age Group, normalized by pop segment size')
ax.set_ylabel('Fraction of population segment')

So after adjusting for the population segment size the mortality rate for women makes a large jump in the 80 and over category.